In [2]:
from __future__ import print_function
from __future__ import division
from gensim.models import Doc2Vec, Word2Vec
import lib.text_extraction as te
from functools import partial
import lib.embedding as em
import lib.words as wd
import pandas as pd
import numpy as np
import operator
import codecs
import math
import json
import csv
import os
import re

Using TensorFlow backend.


## Sentencing

In [2]:
test_dir = '../extraction/esempi_descrizioni/'

def list_dir(d):
    return [os.path.join(test_dir, f) for f in os.listdir(d)]

filenames = os.listdir(test_dir)
full_filenames = list_dir(test_dir)

txts = [te.extract_text(f, do_ocr=True, pages=-1) for f in full_filenames]


../extraction/esempi_descrizioni/5115873790001.pdf Estrazione testo con Vision API.
../extraction/esempi_descrizioni/02628680346_2081942240001.pdf Estrazione testo con Vision API.


In [ ]:
for i, txt in zip(filenames, txts):
    print('Documento ', i)
    print(txt[:1000])

In [3]:
extracted_txts_dir = '../extraction/esempi_extracted/'

for fn, txt in zip(filenames, txts):
    utxt = wd.to_utf8(txt)
    with codecs.open(os.path.join(extracted_txts_dir, fn[:-3])+'txt', 'w', encoding='utf-8') as o:
        o.write(utxt)


In [4]:
sentences_dir = '../extraction/esempi_sentences/'

sentences = [wd.sentences_doc(txt) for txt in txts]

for fn, txt in zip(filenames, sentences):
    with codecs.open(os.path.join(sentences_dir, fn[:-3])+'txt', 'w', encoding='utf-8') as o:
        o.write('\n'.join(s.replace('\n',' ').strip() for s in txt if s.strip()))

## Saving sentences

In [5]:
sentenced_txts = [wd.sentences_doc(txt) for txt in txts]

In [6]:
def sentence_label_csv_empty(filenames, sentenced_txts, csv_out, sep=u'\t', min_len=10):
    csv_out.write(sep.join([u'filename',u'sent_index',u'sentence',u'label']) + u'\n')
    for f, sents in zip(filenames, sentenced_txts):
        for i, sent in enumerate(sents):
            clean_sent = sent.replace(sep,u'').replace(u'\n',u' ').strip()
            if len(clean_sent)>=min_len:
                csv_out.write(sep.join([u'{}',u'{}',u'{}',u'non_rilevante']).format(f,i,clean_sent)+u'\n')
    

In [7]:
csv_filename = '../extraction/sentence_labels.csv'
with codecs.open(csv_filename, 'w', encoding='utf-8') as csv_out:
    sentence_label_csv_empty(os.listdir(test_dir), sentenced_txts, csv_out)

In [8]:
df = pd.read_csv(csv_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

In [9]:
df

,filename,sent_index,sentence,label
0,5120119780001.pdf,0,^ E D d D d & Repertorio n 72928 Raccolta n 10...,non_rilevante
1,5120119780001.pdf,1,Repubblica Italiana Il ventuno giugno duemilad...,non_rilevante
2,5120119780001.pdf,2,"Detto signor comparente, della identità person...",non_rilevante
3,5120119780001.pdf,3,- 2 Sede La società ha sede nel Comune di Torino,non_rilevante
4,5120119780001.pdf,4,L'indirizzo della stessa è in corso Galileo Fe...,non_rilevante
5,5120119780001.pdf,5,"- l'acquisto, la vendita, la permuta e la gest...",non_rilevante
6,5120119780001.pdf,6,"- la lottizzazione di aree residenziali, indus...",non_rilevante
7,5120119780001.pdf,7,- la formazione di comparti secondo le normati...,non_rilevante
8,5120119780001.pdf,8,la partecipazione alla costituzione di consorz...,non_rilevante
9,5120119780001.pdf,9,la stipula di convenzioni ed atti d'obbligo pe...,non_rilevante


## Experiments on labelled sentences

In [16]:
labeled_csv_filename = '../extraction/sentence_manual_labels.csv'

ldf = pd.read_csv(labeled_csv_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

In [17]:
poteri = ldf.loc[ldf['label'] == 'poteri']['sentence']

In [32]:
split_poteri = [[w.lower() for w in wd.splitted_words_utf8(s)] for s in poteri]

In [19]:
def word_counts(sentences):
    d = dict()
    for sentence in sentences:
        for word in sentence:
            if not word in d:
                d[word] = 1
            else:
                d[word] += 1
    return d

def first_n_words(sentences, n):
    wc = word_counts(sentences)
    sorted_wc = sorted(wc.items(), key=operator.itemgetter(1))
    return list(reversed([x for x in sorted_wc[-n:]]))


In [35]:
first_n_words(split_poteri, 50)

[(u'di', 44),
 (u'e', 31),
 (u'poteri', 27),
 (u'per', 24),
 (u'in', 23),
 (u'dei', 23),
 (u'della', 22),
 (u'o', 19),
 (u'i', 19),
 (u'la', 18),
 (u'il', 17),
 (u'amministrazione', 15),
 (u'del', 14),
 (u'che', 14),
 (u'straordinaria', 13),
 (u'con', 13),
 (u'societ\xe0', 13),
 (u'pi\xf9', 12),
 (u'tutti', 12),
 (u'ordinaria', 12),
 (u'legge', 11),
 (u'soci', 10),
 (u'organo', 10),
 (u'amministrativo', 10),
 (u'presente', 10),
 (u'ad', 9),
 (u'suoi', 9),
 (u'ed', 9),
 (u'atto', 8),
 (u'l', 8),
 (u'dall', 7),
 (u'al', 7),
 (u'ai', 7),
 (u'ampi', 7),
 (u'delle', 7),
 (u'\xe8', 7),
 (u'dell', 7),
 (u'Il', 6),
 (u'componenti', 6),
 (u'dalla', 6),
 (u'articolo', 6),
 (u'statuto', 6),
 (u'pu\xf2', 6),
 (u'L', 6),
 (u'le', 6),
 (u'sociale', 6),
 (u'gestione', 6),
 (u'atti', 5),
 (u'quanto', 5),
 (u'facolt\xe0', 5)]

## Tf Idf


In [28]:
def tf(word, sentence):
    '''Term frequency in the sentence'''
    return sum(1 for w in sentence if w == word)/len(sentence)

def idf(word, documents):
    '''Inverse document frequency'''
    D = len(documents)
    den = 1+sum(1 for sentence in documents if word in sentence)
    return math.log(D/den)

def idf_memo(documents):
    '''Defines a memoized version of idf with the given set of documents'''
    word_idf = {}
    def memoized_idf(word):
        if not word in word_idf:
            word_idf[word] = idf(word, documents)
        return word_idf[word]
    return memoized_idf

def tf_idf(word, sentence, sentences):
    '''Simple tf-idf index'''
    return tf(word, sentence)*idf(word, sentences)

def tf_idf_memo(documents):
    '''Defines a memoized version of tf-idf with the given set of documents'''
    idf_memoized = idf_memo(documents)
    def tf_idf_memoized(word, sentence):
        return tf(word, sentence)*idf_memoized(word)
    return tf_idf_memoized
    

In [33]:
split_sentences = [[w.lower() for w in wd.splitted_words_utf8(s)] for s in ldf['sentence']]

tf_idf_memoized = tf_idf_memo(split_sentences)

In [57]:
poteri_words = set(word for sent in split_poteri for word in sent)

poteri_words_mean_tfidf = [(word, np.mean([tf_idf_memoized(word, s) for s in split_poteri])) for word in poteri_words]

In [58]:
sorted_poteri_words = sorted(poteri_words_mean_tfidf, key=operator.itemgetter(1), reverse=True)

In [59]:
print(u'{:20}\t{:20}\n'.format(u'parola',u'mean tf-idf'))
for w, score in sorted_poteri_words:
    print(u'{:20}\t{:1.4f}'.format(w,score))

parola              	mean tf-idf         

poteri              	0.1284
straordinaria       	0.0673
ordinaria           	0.0660
amministrativo      	0.0476
organo              	0.0426
amministrazione     	0.0425
ampi                	0.0395
tutti               	0.0390
atti                	0.0370
i                   	0.0327
per                 	0.0312
dei                 	0.0281
più                 	0.0279
o                   	0.0273
eccezione           	0.0261
società             	0.0246
suoi                	0.0240
della               	0.0236
articolo            	0.0228
amministratori      	0.0221
gestione            	0.0220
determinandone      	0.0218
investito           	0.0213
facoltà             	0.0210
l                   	0.0210
sede                	0.0208
institori           	0.0207
legge               	0.0206
e                   	0.0206
compimento          	0.0202
direttori           	0.0198
limiti              	0.0196
consiglio           	0.0195
procuratori         	0.0191
nomin

In [60]:
most_relevant_words = sorted_poteri_words[:8]

## Embeddings test

In [ ]:
txts_tokenized = [wd.tokenize_doc(txt) for txt in txts]

In [ ]:
reduced_dictionary_filename = 'first_5000_words.json'
with open(reduced_dictionary_filename) as f:
    reduced_dictionary = set(json.load(f))

gensim_model_filename = 'models/gensim_model_5000.d2v'
gensim_model = Doc2Vec.load(gensim_model_filename)

In [ ]:
embeddings = [em.embed_document(gensim_model, txt, reduced_dictionary) for txt in txts_tokenized]

In [ ]:
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


In [ ]:
sv = partial(em.sentence_vector, model=gensim_model, permitted_words=reduced_dictionary)

In [ ]:
p1 = u'All\'organo amministrativo sono conferiti i più ampi poteri, sia per la gestione ordinaria che straordinaria della Società'
p2 = u'ARTICOLO 19 - POTERI DELL\'ORGANO AMMINISTRATIVO 191 L\'organo amministrativo ha tutti i poteri di ordinaria e straordinaria amministrazione'
cosine_similarity(sv(sentence=p1.lower()),sv(sentence=p2.lower())) 

In [ ]:
mean_cosines_poteri = [np.mean([cosine_similarity(sv(sentence=split_poteri[i]), sv(sentence=split_poteri[j]))
                               for i in range(len(split_poteri)) if i != j])
                                   for j in range(len(split_poteri))]

In [ ]:
mean_cosines_poteri

## Word2Vec


In [ ]:
sentences_tokenized = [sent for doc in txts_tokenized for sent in doc] #i should reduce the dictionary also

In [ ]:
w2v = Word2Vec(sentences_tokenized, size=100, window=5, min_count=5, workers=4)

In [ ]:
w2v.wv['costitutivo']

In [ ]:
cosine_similarity(w2v.wv['atto'], w2v.wv['costitutivo'])

In [ ]:
w2v.most_similar('poteri')

In [ ]:
w2v.most_similar(positive=['atto','costitutivo'])